# © Artur Czarnecki. All rights reserved.
# Intergrax framework – proprietary and confidential.
# Use, modification, or distribution without written permission is prohibited.

# Intergrax – StepExecutor Integration Tests

This notebook validates **real StepExecutor behavior** in isolation:
- sequential execution
- dependencies
- retry policies
- controlled replan
- handler contract enforcement
- final output selection

Goal: confirm StepExecutor is stable **before** wiring it into runtime.

In [1]:
import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..", "..")))

In [2]:
from dataclasses import dataclass
from typing import Any, Dict, Mapping

from intergrax.runtime.nexus.planning.stepplan_models import (
    ExecutionStep,
    StepId,
    StepAction,
)

from intergrax.runtime.nexus.planning.step_executor import StepExecutor

from intergrax.runtime.nexus.planning.step_executor_models import (
    ReplanCode,
    ReplanReason,
    StepExecutionContext,
    StepExecutionResult,
    StepStatus,
    StepHandlerRegistry,
    StepExecutorConfig,
    StepReplanRequested,
)

@dataclass
class TestCtx:
    _state: Dict[str, Any]
    _results: Dict[StepId, StepExecutionResult]

    @property
    def state(self) -> Dict[str, Any]:
        return self._state

    @property
    def results(self) -> Mapping[StepId, StepExecutionResult]:
        return self._results

    def set_result(self, result: StepExecutionResult) -> None:
        self._results[result.step_id] = result

async def h_draft(step: ExecutionStep, ctx: StepExecutionContext) -> StepExecutionResult:
    n = int(ctx.state.get("fail_draft_times", 0))
    if n > 0:
        ctx.state["fail_draft_times"] = n - 1
        raise RuntimeError("draft transient failure")

    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        output={"draft": "hello"},
        error=None,
        attempts=1,
    )

async def h_verify(step: ExecutionStep, ctx: StepExecutionContext) -> StepExecutionResult:
    if bool(ctx.state.get("verify_should_fail", False)):
        raise RuntimeError("verification failed")

    dep = ctx.results.get(StepId.DRAFT)
    if dep is None or dep.output is None:
        raise RuntimeError("missing draft dependency output")

    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        output={"verified": True},
        error=None,
        attempts=1,
    )

async def h_final(step: ExecutionStep, ctx: StepExecutionContext) -> StepExecutionResult:
    if bool(ctx.state.get("force_replan", False)):
        raise StepReplanRequested(code=ReplanCode.STEP_POLICY_REPLAN, reason=ReplanReason.UNSPECIFIED)

    if bool(ctx.state.get("mismatch_result", False)):
        return StepExecutionResult(
            step_id=StepId.VERIFY,  # wrong on purpose
            action=step.action,
            status=StepStatus.OK,
            output={"answer": "bad"},
            error=None,
            attempts=1,
        )

    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        output={"answer": "final"},
        error=None,
        attempts=1,
    )

REGISTRY = StepHandlerRegistry(
    handlers={
        StepAction.SYNTHESIZE_DRAFT: h_draft,
        StepAction.VERIFY_ANSWER: h_verify,
        StepAction.FINALIZE_ANSWER: h_final,
    }
)

EXECUTOR = StepExecutor(
    registry=REGISTRY,
    cfg=StepExecutorConfig(
        max_attempts_hard_cap=3,
        fail_fast=True,
    ),
)
print("Configured StepExecutor.")

Configured StepExecutor.


## Test 1 — Happy path (DRAFT → VERIFY → FINAL)

Expect:
- all steps execute successfully
- `ok=True`
- final output is produced
- results contain DRAFT, VERIFY, FINAL

In [3]:
import uuid
from intergrax.runtime.nexus.planning.engine_plan_models import PlanIntent
from intergrax.runtime.nexus.planning.step_executor_models import PlanExecutionReport
from intergrax.runtime.nexus.planning.stepplan_models import (
    ExecutionPlan,
    ExecutionStep,
    PlanBudgets,
    PlanMode,
    StepId,
    StepAction,
    ExpectedOutputType,
    RationaleType,
    StopConditions,
    VerifySeverity,
)

plan = ExecutionPlan(
    plan_id=f"nb-{uuid.uuid4().hex}",
    intent=PlanIntent.GENERIC,
    mode=PlanMode.EXECUTE,
    budgets=PlanBudgets(
        max_total_steps=6,
        max_total_tool_calls=0,
        max_total_web_queries=0,
        max_total_chars_context=12000,
        max_total_tokens_output=None,
    ),
    stop_conditions=StopConditions(
        max_iterations=20,
        stop_on_no_progress=True,
        stop_on_clarifying_question_answered=True,
        stop_on_verifier_pass=True,
        stop_on_budget_exhausted=True,
    ),
    steps=[
        ExecutionStep(
            step_id=StepId.DRAFT,
            action=StepAction.SYNTHESIZE_DRAFT,
            depends_on=[],
            params={
                "instructions": "Produce a short draft answer for the user question.",
                "must_include": [],
                "avoid": [],
            },
            expected_output_type=ExpectedOutputType.DRAFT,
            rationale_type=RationaleType.PRODUCE_DRAFT,
        ),
        ExecutionStep(
            step_id=StepId.VERIFY,
            action=StepAction.VERIFY_ANSWER,
            depends_on=[StepId.DRAFT],
            params={
                "criteria": [
                    {
                        "id": "coherence",
                        "description": "Answer is coherent and addresses the question.",
                        "severity": VerifySeverity.ERROR,
                    }
                ],
                "strict": True,
            },
            expected_output_type=ExpectedOutputType.VERIFIED,
            rationale_type=RationaleType.VERIFY_QUALITY,
        ),
        ExecutionStep(
            step_id=StepId.FINAL,
            action=StepAction.FINALIZE_ANSWER,
            depends_on=[StepId.VERIFY],
            params={
                "instructions": "Finalize the answer. Keep it concise and correct.",
                "format": "markdown",
            },
            expected_output_type=ExpectedOutputType.FINAL,
            rationale_type=RationaleType.FINALIZE,
        ),
    ],
)

result = await EXECUTOR.execute(plan=plan, state={})

print("ok:", result.ok)
print("final_output:", result.final_output)
print("steps:", list(result.step_results.keys()))


ok: True
final_output: {'answer': 'final'}
steps: [<StepId.DRAFT: 'draft'>, <StepId.VERIFY: 'verify'>, <StepId.FINAL: 'final'>]


## Test 2 — Retry on transient handler failure (DRAFT fails once)

Expect:
- DRAFT fails on first attempt, succeeds on second
- overall `ok=True`
- `attempts` for DRAFT > 1


In [4]:
import uuid

from intergrax.runtime.nexus.planning.engine_plan_models import PlanIntent
from intergrax.runtime.nexus.planning.stepplan_models import (
    ExecutionPlan,
    ExecutionStep,
    ExpectedOutputType,
    PlanBudgets,
    PlanMode,
    RationaleType,
    StopConditions,
    StepAction,
    StepId,
    VerifySeverity,
)

state = {"fail_draft_times": 1}

plan = ExecutionPlan(
    plan_id=f"nb-{uuid.uuid4().hex}",
    intent=PlanIntent.GENERIC,
    mode=PlanMode.EXECUTE,
    budgets=PlanBudgets(
        max_total_steps=6,
        max_total_tool_calls=0,
        max_total_web_queries=0,
        max_total_chars_context=12000,
        max_total_tokens_output=None,
    ),
    stop_conditions=StopConditions(
        max_iterations=20,
        stop_on_no_progress=True,
        stop_on_clarifying_question_answered=True,
        stop_on_verifier_pass=True,
        stop_on_budget_exhausted=True,
    ),
    steps=[
        ExecutionStep(
            step_id=StepId.DRAFT,
            action=StepAction.SYNTHESIZE_DRAFT,
            depends_on=[],
            params={
                "instructions": "Produce a short draft answer for the user question.",
                "must_include": [],
                "avoid": [],
            },
            expected_output_type=ExpectedOutputType.DRAFT,
            rationale_type=RationaleType.PRODUCE_DRAFT,
        ),
        ExecutionStep(
            step_id=StepId.VERIFY,
            action=StepAction.VERIFY_ANSWER,
            depends_on=[StepId.DRAFT],
            params={
                "criteria": [
                    {
                        "id": "coherence",
                        "description": "Answer is coherent and addresses the question.",
                        "severity": VerifySeverity.ERROR,
                    }
                ],
                "strict": True,
            },
            expected_output_type=ExpectedOutputType.VERIFIED,
            rationale_type=RationaleType.VERIFY_QUALITY,
        ),
        ExecutionStep(
            step_id=StepId.FINAL,
            action=StepAction.FINALIZE_ANSWER,
            depends_on=[StepId.VERIFY],
            params={
                "instructions": "Finalize the answer. Keep it concise and correct.",
                "format": "markdown",
            },
            expected_output_type=ExpectedOutputType.FINAL,
            rationale_type=RationaleType.FINALIZE,
        ),
    ],
)

result = await EXECUTOR.execute(plan=plan, state=state)

draft_res = result.step_results[StepId.DRAFT]

print("ok:", result.ok)
print("final_output:", result.final_output)
print("draft_attempts:", draft_res.attempts)
print("draft_status:", getattr(draft_res.status, "value", draft_res.status))
print("draft_error_code:", getattr(getattr(draft_res.error, "code", None), "value", getattr(draft_res.error, "code", None)))

assert result.ok is True, result
assert result.final_output is not None, result
assert draft_res.attempts >= 2, draft_res


ok: True
final_output: {'answer': 'final'}
draft_attempts: 2
draft_status: ok
draft_error_code: None


## Test 3 — Replan request (handler raises StepReplanRequested)

Expect:
- executor stops the plan early
- `ok=False` (no final output)
- report indicates replan was requested


In [5]:
import uuid

from intergrax.runtime.nexus.planning.engine_plan_models import PlanIntent
from intergrax.runtime.nexus.planning.stepplan_models import (
    ExecutionPlan,
    ExecutionStep,
    ExpectedOutputType,
    PlanBudgets,
    PlanMode,
    RationaleType,
    StopConditions,
    StepAction,
    StepId,
    VerifySeverity,
)

state = {"force_replan": True}

plan = ExecutionPlan(
    plan_id=f"nb-{uuid.uuid4().hex}",
    intent=PlanIntent.GENERIC,
    mode=PlanMode.EXECUTE,
    budgets=PlanBudgets(
        max_total_steps=6,
        max_total_tool_calls=0,
        max_total_web_queries=0,
        max_total_chars_context=12000,
        max_total_tokens_output=None,
    ),
    stop_conditions=StopConditions(
        max_iterations=20,
        stop_on_no_progress=True,
        stop_on_clarifying_question_answered=True,
        stop_on_verifier_pass=True,
        stop_on_budget_exhausted=True,
    ),
    steps=[
        ExecutionStep(
            step_id=StepId.DRAFT,
            action=StepAction.SYNTHESIZE_DRAFT,
            depends_on=[],
            params={
                "instructions": "Produce a short draft answer for the user question.",
                "must_include": [],
                "avoid": [],
            },
            expected_output_type=ExpectedOutputType.DRAFT,
            rationale_type=RationaleType.PRODUCE_DRAFT,
        ),
        ExecutionStep(
            step_id=StepId.VERIFY,
            action=StepAction.VERIFY_ANSWER,
            depends_on=[StepId.DRAFT],
            params={
                "criteria": [
                    {
                        "id": "coherence",
                        "description": "Answer is coherent and addresses the question.",
                        "severity": VerifySeverity.ERROR,
                    }
                ],
                "strict": True,
            },
            expected_output_type=ExpectedOutputType.VERIFIED,
            rationale_type=RationaleType.VERIFY_QUALITY,
        ),
        ExecutionStep(
            step_id=StepId.FINAL,
            action=StepAction.FINALIZE_ANSWER,
            depends_on=[StepId.VERIFY],
            params={
                "instructions": "Finalize the answer. Keep it concise and correct.",
                "format": "markdown",
            },
            expected_output_type=ExpectedOutputType.FINAL,
            rationale_type=RationaleType.FINALIZE,
        ),
    ],
)

result = await EXECUTOR.execute(plan=plan, state=state)

final_res = result.step_results[StepId.FINAL]

print("ok:", result.ok)
print("final_output:", result.final_output)
print("statuses:", {k: getattr(v.status, "value", v.status) for k, v in result.step_results.items()})
print("final_error_code:", getattr(getattr(final_res.error, "code", None), "value", getattr(final_res.error, "code", None)))
print("final_error_message:", getattr(final_res.error, "message", None))

assert result.ok is False, result
assert result.final_output is None, result
assert getattr(final_res.status, "value", final_res.status) == "replan_requested", final_res
assert final_res.error is not None, final_res


ok: False
final_output: None
statuses: {<StepId.DRAFT: 'draft'>: 'ok', <StepId.VERIFY: 'verify'>: 'ok', <StepId.FINAL: 'final'>: 'replan_requested'}
final_error_code: replan
final_error_message: step_policy_replan: unspecified


## Test 4 — Dependency propagation (VERIFY and FINAL consume previous outputs)

Expect:
- DRAFT produces a draft output
- VERIFY reads DRAFT output and produces verified=True
- FINAL reads VERIFY output and produces final answer
- ok=True and final_output is present


In [6]:
# Test 4 — Dependency propagation (VERIFY and FINAL consume previous outputs)

import uuid

from intergrax.runtime.nexus.planning.engine_plan_models import PlanIntent
from intergrax.runtime.nexus.planning.step_executor import StepExecutor
from intergrax.runtime.nexus.planning.step_executor_models import (
    StepError,
    StepErrorCode,
    StepExecutionResult,
    StepExecutorConfig,
    StepHandlerRegistry,
    StepStatus,
)
from intergrax.runtime.nexus.planning.stepplan_models import (
    ExecutionPlan,
    ExecutionStep,
    ExpectedOutputType,
    PlanBudgets,
    PlanMode,
    RationaleType,
    StopConditions,
    StepAction,
    StepId,
    VerifySeverity,
)

# Handlers deliberately consume dependency outputs from ctx.results

async def h_draft(step: ExecutionStep, ctx) -> StepExecutionResult:
    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        output={"draft": "draft"},
        error=None,
        attempts=1,
    )

async def h_verify(step: ExecutionStep, ctx) -> StepExecutionResult:
    draft_res = ctx.results.get(StepId.DRAFT)
    draft_out = None if draft_res is None else draft_res.output

    if not isinstance(draft_out, dict) or "draft" not in draft_out:
        return StepExecutionResult(
            step_id=step.step_id,
            action=step.action,
            status=StepStatus.FAILED,
            output=None,
            error=StepError(
                code=StepErrorCode.DEPENDENCY_MISSING,
                message="Missing/invalid DRAFT output in dependency propagation test.",
                details={"depends_on": [StepId.DRAFT.value]},
            ),
            attempts=1,
        )

    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        output={"verified": True, "seen_draft": draft_out["draft"]},
        error=None,
        attempts=1,
    )

async def h_final(step: ExecutionStep, ctx) -> StepExecutionResult:
    verify_res = ctx.results.get(StepId.VERIFY)
    verify_out = None if verify_res is None else verify_res.output

    if not isinstance(verify_out, dict) or verify_out.get("verified") is not True:
        return StepExecutionResult(
            step_id=step.step_id,
            action=step.action,
            status=StepStatus.FAILED,
            output=None,
            error=StepError(
                code=StepErrorCode.DEPENDENCY_MISSING,
                message="Missing/invalid VERIFY output in dependency propagation test.",
                details={"depends_on": [StepId.VERIFY.value]},
            ),
            attempts=1,
        )

    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        output={"answer": "final"},
        error=None,
        attempts=1,
    )

REGISTRY = StepHandlerRegistry(
    handlers={
        StepAction.SYNTHESIZE_DRAFT: h_draft,
        StepAction.VERIFY_ANSWER: h_verify,
        StepAction.FINALIZE_ANSWER: h_final,
    }
)

EXECUTOR = StepExecutor(
    registry=REGISTRY,
    cfg=StepExecutorConfig(
        max_attempts_hard_cap=3,
        fail_fast=True,
    ),
)

state = {}

plan = ExecutionPlan(
    plan_id=f"nb-{uuid.uuid4().hex}",
    intent=PlanIntent.GENERIC,
    mode=PlanMode.EXECUTE,
    budgets=PlanBudgets(
        max_total_steps=6,
        max_total_tool_calls=0,
        max_total_web_queries=0,
        max_total_chars_context=12000,
        max_total_tokens_output=None,
    ),
    stop_conditions=StopConditions(
        max_iterations=20,
        stop_on_no_progress=True,
        stop_on_clarifying_question_answered=True,
        stop_on_verifier_pass=True,
        stop_on_budget_exhausted=True,
    ),
    steps=[
        ExecutionStep(
            step_id=StepId.DRAFT,
            action=StepAction.SYNTHESIZE_DRAFT,
            depends_on=[],
            params={
                "instructions": "Produce a short draft answer for the user question.",
                "must_include": [],
                "avoid": [],
            },
            expected_output_type=ExpectedOutputType.DRAFT,
            rationale_type=RationaleType.PRODUCE_DRAFT,
        ),
        ExecutionStep(
            step_id=StepId.VERIFY,
            action=StepAction.VERIFY_ANSWER,
            depends_on=[StepId.DRAFT],
            params={
                "criteria": [
                    {
                        "id": "coherence",
                        "description": "Answer is coherent and addresses the question.",
                        "severity": VerifySeverity.ERROR,
                    }
                ],
                "strict": True,
            },
            expected_output_type=ExpectedOutputType.VERIFIED,
            rationale_type=RationaleType.VERIFY_QUALITY,
        ),
        ExecutionStep(
            step_id=StepId.FINAL,
            action=StepAction.FINALIZE_ANSWER,
            depends_on=[StepId.VERIFY],
            params={
                "instructions": "Finalize the answer. Keep it concise and correct.",
                "format": "markdown",
            },
            expected_output_type=ExpectedOutputType.FINAL,
            rationale_type=RationaleType.FINALIZE,
        ),
    ],
)

result = await EXECUTOR.execute(plan=plan, state=state)

print("ok:", result.ok)
print("final_output:", result.final_output)
print("steps:", list(result.step_results.keys()))
print("statuses:", {k: v.status.value for k, v in result.step_results.items()})
print("verify_output:", result.step_results[StepId.VERIFY].output)

assert result.ok is True, result
assert result.final_output == {"answer": "final"}, result.final_output
assert result.step_results[StepId.VERIFY].output == {"verified": True, "seen_draft": "draft"}


ok: True
final_output: {'answer': 'final'}
steps: [<StepId.DRAFT: 'draft'>, <StepId.VERIFY: 'verify'>, <StepId.FINAL: 'final'>]
statuses: {<StepId.DRAFT: 'draft'>: 'ok', <StepId.VERIFY: 'verify'>: 'ok', <StepId.FINAL: 'final'>: 'ok'}
verify_output: {'verified': True, 'seen_draft': 'draft'}


## Test 5 — Dependency failure (VERIFY fails when DRAFT output is missing)

Expect:
- DRAFT returns invalid output
- VERIFY fails with DEPENDENCY_MISSING
- FINAL is not executed (or plan.ok=False)
- final_output is None


In [7]:
# Test 5 — Dependency failure (VERIFY fails when DRAFT output is missing)

import uuid

from intergrax.runtime.nexus.planning.engine_plan_models import PlanIntent
from intergrax.runtime.nexus.planning.step_executor import StepExecutor
from intergrax.runtime.nexus.planning.step_executor_models import (
    StepError,
    StepErrorCode,
    StepExecutionResult,
    StepExecutorConfig,
    StepHandlerRegistry,
    StepStatus,
)
from intergrax.runtime.nexus.planning.stepplan_models import (
    ExecutionPlan,
    ExecutionStep,
    ExpectedOutputType,
    PlanBudgets,
    PlanMode,
    RationaleType,
    StopConditions,
    StepAction,
    StepId,
    VerifySeverity,
)

# Same handlers as Test 4, but DRAFT returns invalid output (None)

async def h_draft(step: ExecutionStep, ctx) -> StepExecutionResult:
    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        output=None,  # invalid for downstream dependency
        error=None,
        attempts=1,
    )

async def h_verify(step: ExecutionStep, ctx) -> StepExecutionResult:
    draft_res = ctx.results.get(StepId.DRAFT)
    draft_out = None if draft_res is None else draft_res.output

    if not isinstance(draft_out, dict) or "draft" not in draft_out:
        return StepExecutionResult(
            step_id=step.step_id,
            action=step.action,
            status=StepStatus.FAILED,
            output=None,
            error=StepError(
                code=StepErrorCode.DEPENDENCY_MISSING,
                message="Missing/invalid DRAFT output (expected dict with 'draft').",
                details={"depends_on": [StepId.DRAFT.value]},
            ),
            attempts=1,
        )

    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        output={"verified": True},
        error=None,
        attempts=1,
    )

async def h_final(step: ExecutionStep, ctx) -> StepExecutionResult:
    # Should not be executed in this test if fail_fast works as expected
    return StepExecutionResult(
        step_id=step.step_id,
        action=step.action,
        status=StepStatus.OK,
        output={"answer": "final"},
        error=None,
        attempts=1,
    )

REGISTRY = StepHandlerRegistry(
    handlers={
        StepAction.SYNTHESIZE_DRAFT: h_draft,
        StepAction.VERIFY_ANSWER: h_verify,
        StepAction.FINALIZE_ANSWER: h_final,
    }
)

EXECUTOR = StepExecutor(
    registry=REGISTRY,
    cfg=StepExecutorConfig(
        max_attempts_hard_cap=3,
        fail_fast=True,
    ),
)

state = {}

plan = ExecutionPlan(
    plan_id=f"nb-{uuid.uuid4().hex}",
    intent=PlanIntent.GENERIC,
    mode=PlanMode.EXECUTE,
    budgets=PlanBudgets(
        max_total_steps=6,
        max_total_tool_calls=0,
        max_total_web_queries=0,
        max_total_chars_context=12000,
        max_total_tokens_output=None,
    ),
    stop_conditions=StopConditions(
        max_iterations=20,
        stop_on_no_progress=True,
        stop_on_clarifying_question_answered=True,
        stop_on_verifier_pass=True,
        stop_on_budget_exhausted=True,
    ),
    steps=[
        ExecutionStep(
            step_id=StepId.DRAFT,
            action=StepAction.SYNTHESIZE_DRAFT,
            depends_on=[],
            params={
                "instructions": "Produce a short draft answer for the user question.",
                "must_include": [],
                "avoid": [],
            },
            expected_output_type=ExpectedOutputType.DRAFT,
            rationale_type=RationaleType.PRODUCE_DRAFT,
        ),
        ExecutionStep(
            step_id=StepId.VERIFY,
            action=StepAction.VERIFY_ANSWER,
            depends_on=[StepId.DRAFT],
            params={
                "criteria": [
                    {
                        "id": "coherence",
                        "description": "Answer is coherent and addresses the question.",
                        "severity": VerifySeverity.ERROR,
                    }
                ],
                "strict": True,
            },
            expected_output_type=ExpectedOutputType.VERIFIED,
            rationale_type=RationaleType.VERIFY_QUALITY,
        ),
        ExecutionStep(
            step_id=StepId.FINAL,
            action=StepAction.FINALIZE_ANSWER,
            depends_on=[StepId.VERIFY],
            params={
                "instructions": "Finalize the answer. Keep it concise and correct.",
                "format": "markdown",
            },
            expected_output_type=ExpectedOutputType.FINAL,
            rationale_type=RationaleType.FINALIZE,
        ),
    ],
)

result = await EXECUTOR.execute(plan=plan, state=state)

print("ok:", result.ok)
print("final_output:", result.final_output)
print("steps:", list(result.step_results.keys()))
print("statuses:", {k: v.status.value for k, v in result.step_results.items()})

verify_res = result.step_results.get(StepId.VERIFY)
print("verify_status:", None if verify_res is None else verify_res.status.value)
print("verify_error_code:", None if verify_res is None else verify_res.error.code.value)
print("verify_error_message:", None if verify_res is None else verify_res.error.message)

assert result.ok is False, result
assert result.final_output is None, result.final_output

# DRAFT ran
assert StepId.DRAFT in result.step_results
# VERIFY ran and failed
assert StepId.VERIFY in result.step_results
assert result.step_results[StepId.VERIFY].status.value == "failed"
assert result.step_results[StepId.VERIFY].error.code.value == "dependency_missing"

# With fail_fast=True, FINAL should not run
assert StepId.FINAL not in result.step_results


ok: False
final_output: None
steps: [<StepId.DRAFT: 'draft'>, <StepId.VERIFY: 'verify'>]
statuses: {<StepId.DRAFT: 'draft'>: 'ok', <StepId.VERIFY: 'verify'>: 'failed'}
verify_status: failed
verify_error_code: dependency_missing
verify_error_message: Missing/invalid DRAFT output (expected dict with 'draft').
